In [ ]:
import pandas as pd
import re
import glob

In [ ]:
# 運用glob取得當前目錄所有txt檔
files = glob.glob("*.txt")

#　Create a function which add on number for grouping each order batch later
def order_with_num(df):
    n = 0
    num_col = []
    for i in df[1]:
        if i[0] == "M":
            n += 1
            num_col.append(n)
        else:
            num_col.append(None)
    return num_col

# 執行code，檢查有無檔案
if len(files)>0:
    for file in files:
        name = file.split(".")[0]
        df = pd.read_csv(file, sep=r"\s+", header=None, names=[1,2,3,4,5,6,7])

        # 將號碼放入dataframe裡面，以front fill方式補上缺值
        df.insert(0, 0, order_with_num(df))
        df[0].fillna(method="ffill", inplace=True)

        # 分離成supplier、product兩個資料，為避免column名稱重複而降低可讀性，將product欄位重新定義
        df_sup = df.loc[df[1].str.contains("M")]
        df_pro = df.loc[df[1].str.contains("D")]
        df_pro.columns = [0, 8, 9, 10, 11, 12, 13, 14]

        # 以訂單編號為鍵，採LeftJoin方式將兩個資料merge在一起
        df_merged = pd.merge(df_sup, df_pro, how="left", on=0)

        # 進行簡單處理後，僅保留需要欄位
        df_merged[15] = df_merged[4].apply(lambda x:x[8:])
        df_merged[4] = df_merged[4].apply(lambda x:x[:8])
        df_merged[8] = df_merged[8].apply(lambda x:x[1:])

        cols = [3,15,4,6,8,12,13]
        df_merged = df_merged[cols]
        df_merged.columns = [u"採購單號碼", u"廠商代碼", u"採購日期", u"送達期限", u"項目代號", u"採購數量", u"單位"]

        # print出結果，確認是否執行成功
        print(df_merged)

        # 輸出成為excel檔
        df_merged.to_excel(f"{name}.xlsx", index=False)

else:
    print("Oops!找不到檔案，請確認是否上傳")